In [ ]:
import polars as pl

# 1. Importar datos desde un archivo Excel
ruta_archivo = "ATENEO BASE PLANA (3).xlsx"
df = pl.read_excel(ruta_archivo, sheet_name=0, skip_rows=3)

# 2. Limpiar espacios en blanco en columnas de tipo string
df = df.with_columns([
    pl.col(col).str.strip().alias(col) if pl.col(col).dtype == pl.Utf8 else pl.col(col)
    for col in df.columns
])

# 3. Filtrar los datos del laboratorio "Siegfried"
df_siegfried = df.filter(pl.col("Laboratorio").str.strip() == "Siegfried")
df_siegfried.describe()

# 4. Hacer merge sobre el df completo
df_completo_type_dosis_mol = df.join(
    df_siegfried.select(["Concentration Type & Factor", "Dosis3", "Molécula(s)", "Precio PVP"]),
    on=["Concentration Type & Factor", "Dosis3", "Molécula(s)"],
    how="left"
).with_column(
    pl.col("Precio PVP").alias("Precio_Siegfried")
)

# Elimina columnas de la 22 a la 88 (índices basados en 0)
columnas_a_eliminar = df.columns[22:89]
df = df.drop(columnas_a_eliminar)

# 5. Eliminar el precio duplicado para "Siegfried"
df_completo_type_dosis_mol = df_completo_type_dosis_mol.with_column(
    pl.when(pl.col("Laboratorio") == "Siegfried")
    .then(None)
    .otherwise(pl.col("Precio_Siegfried"))
    .alias("Precio_Siegfried")
)

# 6. Guardar el resultado en un archivo CSV
df_completo_type_dosis_mol.write_csv("df_completo_type_dosis_mol.csv")